# MLP en Spark

In [1]:
# Import SparkSession
import findspark
findspark.init()

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.sql import SparkSession
import random
import sys
#path = str(sys.argv[1])
#atr = int(sys.argv[2])

In [ ]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[6]") \
    .appName("Data exploration URL") \
    .config("spark.executor.memory", "4gb") \
    .getOrCreate()

sc = spark.sparkContext

In [ ]:
sc._conf.getAll()

In [ ]:
# Load training data
data = spark.read.format("libsvm")\
    .load("../data/url_svmlight/Dimension_General_x_100.svm")
# Split the data into train and test
seed = random.randrange(500, 1300, 2)
splits = data.randomSplit([0.6, 0.4], seed)

train = splits[0]
test = splits[1]

In [ ]:
data.take(1)

In [10]:
# specify layers for the neural network:
# input layer of size 4 (features), two intermediate of size 5 and 4
# and output of size 3 (classes)
layers = [100, 5, 4, 2]

# create the trainer and set its parameters
trainer = MultilayerPerceptronClassifier(
    maxIter=1000, layers=layers, blockSize=1000, seed=1234)

# train the model
model = trainer.fit(train)

In [11]:
# compute accuracy on the test set
result = model.transform(test)
predictionAndLabels = result.select("prediction", "label")

In [12]:
evaluator = MulticlassClassificationEvaluator(metricName="f1")
print("f1: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="weightedPrecision")
print("weightedPrecision: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
print("weightedRecall: " + str(evaluator.evaluate(predictionAndLabels)))
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
print("Accuracy: " + str(evaluator.evaluate(predictionAndLabels)))

f1: 0.9750246998082348
weightedPrecision: 0.9750231797207569
weightedRecall: 0.9750262793452689
Accuracy: 0.9750262793452689


In [9]:
x = 1

In [10]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import StringType
from pyspark import SQLContext
from pyspark.sql import functions as F
conf = SparkConf().setMaster("local[11]").setAppName("KDD")
conf.set("spark.driver.memory", "28g") 
sc=SparkContext(conf=conf)
sqlContext=SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Data exploration URL, master=local[6]) created by getOrCreate at <ipython-input-2-a520991c68d4>:2 

In [ ]:
dfspark=sqlContext.read.format("csv").option("header","true").option("inferSchema","true").load('../data/url_svmlight/prueba.svm')

In [ ]:
dfspark.head()

In [ ]:
dfspark.printSchema()